# Pemrograman Tugas Akhir:


"Segmentasi Burned Area menggunakan Model U-Net pada Citra Landsat 9 (Studi Kasus: Sumatera Selatan)"


*   Model
*   Evaluasi Model



# Library & Mount Drive

In [1]:
!pip install rasterio

In [ ]:
#!pip install opencv-python-headless rasterio scikit-learn tensorflow


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

import os
import cv2
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy


# Load Data

In [3]:
def load_geotiff_data(input_folder, mask_folder, size=(512, 512)):
    X, Y = [], []

    input_files = sorted(os.listdir(input_folder))
    mask_files = sorted(os.listdir(mask_folder))

    if len(input_files) == 0 or len(mask_files) == 0:
        raise ValueError("Folder kosong!")

    if len(input_files) != len(mask_files):
        print(f" Jumlah file berbeda! Input: {len(input_files)}, Mask: {len(mask_files)}")

    for fname_input, fname_mask in zip(input_files, mask_files):
        input_path = os.path.join(input_folder, fname_input)
        mask_path = os.path.join(mask_folder, fname_mask)

        if not (os.path.exists(input_path) and os.path.exists(mask_path)):
            print(f" Lewatkan file (tidak ditemukan): {fname_input}, {fname_mask}")
            continue

        try:
            with rasterio.open(input_path) as src:
                img = src.read()  # (bands, H, W)
                img = np.transpose(img, (1, 2, 0))  # → (H, W, bands)
                img = img.astype(np.float32)
                img = cv2.resize(img, size)
        except:
            print(f" Gagal baca input: {fname_input}")
            continue

        try:
            with rasterio.open(mask_path) as msk:
                mask = msk.read(1)
                mask = mask.astype(np.float32)
                mask = cv2.resize(mask, size)
        except:
            print(f" Gagal baca mask: {fname_mask}")
            continue

        X.append(img)
        Y.append(mask)

    if len(X) == 0:
        raise ValueError("Tidak ada data valid!")

    X = np.array(X)
    Y = np.array(Y)
    Y = Y / 255.0 if Y.max() > 1 else Y
    Y = np.expand_dims(Y, axis=-1)

    print(f"Loaded: {len(X)} data pairs")
    return X, Y


In [4]:
#base_path = "/content/drive/MyDrive/DATA BURNED AREA LANDSAT 9/SPLITDATA" #Google Collab
base_path = '/kaggle/input/data-burned-area-landsat-9-toa/DataBurnedArea'
x_train, y_train = load_geotiff_data(f"{base_path}/TRAIN/MULTIBAND", f"{base_path}/TRAIN/MASK")
x_val, y_val     = load_geotiff_data(f"{base_path}/VAL/MULTIBAND", f"{base_path}/VAL/MASK")
x_test, y_test   = load_geotiff_data(f"{base_path}/TEST/MULTIBAND", f"{base_path}/TEST/MASK")


Loaded: 706 data pairs
Loaded: 151 data pairs
Loaded: 151 data pairs


# Model U-Net

In [5]:
def unet_model(input_size=(512, 512, 5)):
    inputs = layers.Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D(2)(c1)

    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D(2)(c2)

    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D(2)(c3)

    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D(2)(c4)

    #Bottleneck

    c5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, 2, strides=2, padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, 3, activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, 3, activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, 2, strides=2, padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, 3, activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, 3, activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, 2, strides=2, padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, 3, activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, 3, activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, 2, strides=2, padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, 3, activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, 3, activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c9)

    return models.Model(inputs=[inputs], outputs=[outputs])


# Compile Model

In [ ]:
import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Inisialisasi dan kompilasi model
model = unet_model(input_size=x_train.shape[1:])
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Mulai hitung waktu pelatihan
start_time = time.time()

# Training
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    batch_size=8,
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

# Simpan waktu akhir pelatihan
end_time = time.time()


# Informasi GPU dan Waktu Eksekusi

In [ ]:
import GPUtil

# Hitung dan tampilkan waktu pelatihan
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)
print(f" Waktu Training Model: {minutes} menit {seconds} detik")

# Tampilkan detail GPU
gpus = GPUtil.getGPUs()
if gpus:
    print("\n  GPU yang digunakan:")
    for gpu in gpus:
        print(f"    Nama GPU        : {gpu.name}")
        print(f"    ID              : {gpu.id}")
        print(f"    Memory Digunakan: {gpu.memoryUsed}MB / {gpu.memoryTotal}MB")
        print(f"    Load GPU        : {gpu.load * 100:.1f}%")
else:
    print("  Tidak ada GPU terdeteksi. Model dilatih menggunakan CPU.")


# Evaluasi Model

In [ ]:
y_pred = model.predict(x_test)
y_pred_bin = (y_pred > 0.5).astype(np.uint8)
y_true_bin = y_test.astype(np.uint8)

def calculate_metrics_np(y_true, y_pred):
    smooth = 1e-7
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()

    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f)

    dice = (2. * intersection + smooth) / (union + smooth)
    iou = (intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) - intersection + smooth)
    return dice, iou

dice_score, iou_score = calculate_metrics_np(y_true_bin, y_pred_bin)
acc_score = accuracy_score(y_true_bin.flatten(), y_pred_bin.flatten())
bce = BinaryCrossentropy()(y_test, y_pred).numpy()

print(f"Akurasi             : {acc_score:.5f}")
print(f"Dice Coefficient    : {dice_score:.5f}")
print(f"IoU                 : {iou_score:.5f}")
print(f"Binary Crossentropy : {bce:.5f}")


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Hitung confusion matrix
cm = confusion_matrix(y_true_bin.flatten(), y_pred_bin.flatten())

# Tampilkan confusion matrix dengan label jelas
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["Tidak Terbakar", "Terbakar"]  # Ganti label ini sesuai konteks
)
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix - Segmentasi Area Terbakar")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Grafik Loss & Accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend(); plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend(); plt.title("Accuracy")

plt.tight_layout()
plt.show()


# Visualisasi Prediksi

In [ ]:
import matplotlib.pyplot as plt

# Tampilkan semua sample dari x_test
num_samples = len(x_test)
plt.figure(figsize=(15, num_samples * 3))

for i in range(num_samples):
    idx = i

    # Input dengan band 5-4-3 (index 4,3,2)
    rgb_image = x_test[idx][:, :, [4, 3, 2]]

    # Jika perlu normalisasi manual (jika range bukan 0-1)
    rgb_image = (rgb_image - rgb_image.min()) / (rgb_image.max() - rgb_image.min())

    plt.subplot(num_samples, 3, i * 3 + 1)
    plt.imshow(rgb_image)
    plt.title(f"Multiband \nIndex {idx}")
    plt.axis('off')

    plt.subplot(num_samples, 3, i * 3 + 2)
    plt.imshow(y_test[idx].squeeze(), cmap='gray')
    plt.title("Ground Truth")
    plt.axis('off')

    plt.subplot(num_samples, 3, i * 3 + 3)
    plt.imshow(y_pred_bin[idx].squeeze(), cmap='gray')
    plt.title("Prediksi")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.losses import BinaryCrossentropy

# Fungsi evaluasi per citra
def calc_metrics(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()

    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f)

    smooth = 1e-7
    dice = (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)
    iou = (intersection + smooth) / (np.sum(y_true_f + y_pred_f) - intersection + smooth)

    acc = np.mean(y_true_f == y_pred_f)

    bce_fn = BinaryCrossentropy()
    bce = bce_fn(y_true, y_pred).numpy()

    return acc, iou, dice, bce

# Tampilkan semua sample dari x_test
num_samples = len(x_test)
plt.figure(figsize=(20, num_samples * 3))

for i in range(num_samples):
    idx = i

    # Ambil RGB band 5-4-3
    rgb_image = x_test[idx][:, :, [4, 3, 2]]
    rgb_image = (rgb_image - rgb_image.min()) / (rgb_image.max() - rgb_image.min())  # Normalisasi

    y_true = y_test[idx].squeeze()
    y_pred = y_pred_bin[idx].squeeze()

    acc, iou, dice, bce = calc_metrics(y_true, y_pred)

    # Kolom 1 - Input
    plt.subplot(num_samples, 4, i * 4 + 1)
    plt.imshow(rgb_image)
    plt.title(f"Multiband\nIndex {idx}")
    plt.axis('off')

    # Kolom 2 - Ground Truth
    plt.subplot(num_samples, 4, i * 4 + 2)
    plt.imshow(y_true, cmap='gray')
    plt.title("Ground Truth")
    plt.axis('off')

    # Kolom 3 - Prediksi
    plt.subplot(num_samples, 4, i * 4 + 3)
    plt.imshow(y_pred, cmap='gray')
    plt.title("Prediksi")
    plt.axis('off')

    # Kolom 4 - Metrik
    plt.subplot(num_samples, 4, i * 4 + 4)
    plt.axis('off')
    plt.text(0.1, 0.8, f"Accuracy : {acc:.4f}", fontsize=10)
    plt.text(0.1, 0.6, f"IoU     : {iou:.4f}", fontsize=10)
    plt.text(0.1, 0.4, f"Dice Coefficient    : {dice:.4f}", fontsize=10)
    plt.text(0.1, 0.2, f"BCE     : {bce:.4f}", fontsize=10)

plt.tight_layout()
plt.show()
